# Transfer Learning

In [ ]:
# uses this data: https://github.com/mrdbourke/tensorflow-deep-learning/tree/main?tab=readme-ov-file

In [ ]:
import os

for dirthpath, dirnames, filenames in os.walk("../Datasets/10_food_classes_10_percent"):
    print(f"There are {len(dirnames)} directories and {len(filenames)} files in {dirthpath}")

## Creating data loaders (preparing the data)

In [ ]:
# Setup data inputs

from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224,224)
BATCH_SIZE = 32

train_dir = "../Datasets/10_food_classes_10_percent/train"
test_dir = "../Datasets/10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale = 1/255.)

print("Training data:")
train_data = train_datagen.flow_from_directory(directory = train_dir,
                                               target_size = IMAGE_SHAPE,
                                               batch_size = BATCH_SIZE,
                                               class_mode = 'categorical',
                                               )

print("Testing data:")
test_data = test_datagen.flow_from_directory(directory = test_dir,
                                               target_size = IMAGE_SHAPE,
                                               batch_size = BATCH_SIZE,
                                               class_mode = 'categorical',
                                               )

## Setting up callbacks:

Some examples of callbacks:
* Stopping early with EarlyStopingCallback
* Tracking experiments with Tensorboard Callback
* Model Checkpoint with ModelCheckpoint Callback

In [ ]:
# Create a tensorboard callback (functionized because we need to create a new one for each model)

import tensorflow as tf
import datetime

def create_tensorboard_callback(dir_name , experiment):
    log_dir = dir_name + "/" + experiment + "/" + datetime.datetime.now().strftime("%Y%M%D-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.Tensorboard(log_dir = log_dir)
    print(f"Saving TensorBoard log files to {log_dir}")
    return tensorboard_callback

## Creating models using Tensorflow Hub

www.tfhub.dev

In [ ]:
# THESE ARE OLD LINKS FROM THE VIDEOS BUT THEY RE-DIRECT TO THE CORRECT KAGGLE LINKS
# IMPORTANT TO NOTE THAT IT APPEARS THERE ARE BETTER MODELS NOW

# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

# # New: EfficientNetB0 feature vector (version 2)
# efficientnet_url = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2"

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# create a function to make the models:

def create_model(model_url, num_classes = 10):
    """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
    Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

    Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
    """
    # Download the pretrained model and save it as a Keras layer
    feature_extractor_layer = hub.KerasLayer(model_url,
                                            trainable=False, # freeze the underlying patterns
                                            name='feature_extraction_layer',
                                            input_shape=IMAGE_SHAPE+(3,) # define the input image shape
                                            )
  
    # Create our own model
    model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    layers.Dense(num_classes, activation='softmax', name='output_layer') # create our own output layer      
    ])

    return model

### Creating and Testing ResNet Tensorflow Hub Feature Extraction Model

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

print("TensorFlow version:", tf.__version__)
print("TensorFlow Hub version:", hub.__version__)


In [ ]:
# Create Resnet Model

resnet_model = create_model(resnet_url, num_classes= train_data.num_classes)

In [ ]:
resnet_model.summary()